In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from registration_utilities import *
from metadata import *
from data_manager import *

Setting environment for Gordon


No vtk


In [2]:
stack_fixed = 'MD642'
stack_moving = 'atlasV2'

classifier_setting = 2

# Global Transform - between two annotation volumes

In [56]:
global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_global_alignment_parameters(stack_moving=stack_moving, moving_volume_type='annotation',
                                            stack_fixed=stack_fixed, fixed_volume_type='annotation',
                                            global_transform_scheme=global_transform_scheme)

In [84]:
vol_m = DataManager.load_annotation_volume(stack=stack_moving, downscale=32)

volume_m_alignedTo_f = \
transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                  xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

volume_m_alignedTo_f_fn = DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='annotation',
                                        stack_f=stack_fixed, type_f='annotation',
                                        downscale=32,
                                        global_transform_scheme=global_transform_scheme)

create_if_not_exists(os.path.dirname(volume_m_alignedTo_f_fn))
bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)

In [ ]:
display_volume_sections(volume_m_alignedTo_f)

# Global Transform Annotation Volumes

In [21]:
stack_fixed = 'MD589'
stack_moving = 'MD585'

In [22]:
warp_setting = 9

if warp_setting == 1:
    upstream_warp_setting = None
    transform_type = 'affine'
elif warp_setting == 7:
    upstream_warp_setting = None
    transform_type = 'rigid'
elif warp_setting == 8:
    upstream_warp_setting = None
    transform_type = None
elif warp_setting == 9:
    upstream_warp_setting = 8
    transform_type = 'rigid'
else:
    raise Exception('Warp setting not recognized.')

In [23]:
# Load transform parameters

global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                      type_f='annotation_as_score',
                                      type_m='annotation_as_score',
                                      warp_setting=warp_setting,
                                      trial_idx=0)

In [24]:
# for structure in all_known_structures_sided_with_surround:
for structure in all_known_structures_sided:
# for structure in ['7N_L']:
    
    try:
        t = time.time()

        vol_m = DataManager.load_transformed_volume(stack_m=stack_moving, stack_f=stack_fixed,
                                                          type_m='annotation_as_score', 
                                                          type_f='annotation_as_score',
                                                         warp_setting=upstream_warp_setting,
                                                         structure=structure)
        
        volume_m_alignedTo_f = \
        transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                          xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

        volume_m_alignedTo_f_fn = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving, stack_f=stack_fixed,
                                                    type_m='annotation_as_score',
                                                    type_f='annotation_as_score',
                                                    warp_setting=warp_setting,
                                                    structure=structure)

        create_if_not_exists(os.path.dirname(volume_m_alignedTo_f_fn))
        bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)
    
        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t)) # 3s
    
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error transforming volume %s.\n' % structure)

Transform: 1.79 seconds.
Transform: 1.77 seconds.
Transform: 1.76 seconds.
Transform: 1.76 seconds.
Transform: 1.79 seconds.
Transform: 1.79 seconds.
Transform: 1.82 seconds.
Transform: 1.82 seconds.
Transform: 1.76 seconds.
Transform: 1.76 seconds.
Transform: 1.77 seconds.
Transform: 1.76 seconds.
Transform: 1.79 seconds.
Transform: 1.79 seconds.
Transform: 1.81 seconds.
Transform: 1.80 seconds.
Transform: 1.78 seconds.
Transform: 1.78 seconds.
Transform: 1.81 seconds.
Transform: 1.80 seconds.
Transform: 1.78 seconds.
Transform: 1.78 seconds.
Transform: 1.80 seconds.
Transform: 1.81 seconds.
Transform: 1.90 seconds.
Transform: 1.88 seconds.
Transform: 1.77 seconds.
Transform: 1.77 seconds.
Transform: 1.77 seconds.
Transform: 1.77 seconds.
Transform: 1.80 seconds.
Transform: 1.79 seconds.
Transform: 1.78 seconds.
Transform: 1.80 seconds.
Transform: 1.86 seconds.
Transform: 1.85 seconds.
Transform: 1.77 seconds.
Transform: 1.77 seconds.
Transform: 1.78 seconds.
Transform: 1.78 seconds.


# Global Transform

In [3]:
warp_setting = 1
trial_idx = 0

In [4]:
# Load transform parameters

global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                      classifier_setting_m=classifier_setting,
                                      classifier_setting_f=classifier_setting,
                                      warp_setting=warp_setting,
                                      trial_idx=0)

In [6]:
# Transform moving volume, sided, without surround

for structure in all_known_structures_sided_with_surround:
# for structure in all_known_structures_sided:
# for structure in ['7N_L']:
    
    try:
        t = time.time()

        vol_m = DataManager.load_score_volume(stack=stack_moving, structure=structure, downscale=32)

        volume_m_alignedTo_f = \
        transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                          xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

        volume_m_alignedTo_f_fn = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='score',
                                                    stack_f=stack_fixed, type_f='score',
                                                    downscale=32,
                                                    classifier_setting_m=classifier_setting,
                                                    classifier_setting_f=classifier_setting,
                                                    warp_setting=warp_setting,
                                                    structure=structure)

        create_if_not_exists(os.path.dirname(volume_m_alignedTo_f_fn))
        bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)
    
        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t)) # 3s
    
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error transforming volume %s.\n' % structure)

Transform: 3.28 seconds.
Transform: 3.33 seconds.
Transform: 3.26 seconds.
Transform: 3.32 seconds.
Transform: 3.24 seconds.
Transform: 3.38 seconds.
Transform: 3.23 seconds.
Transform: 3.30 seconds.
Transform: 3.24 seconds.
Transform: 3.33 seconds.
Transform: 3.20 seconds.
Transform: 3.28 seconds.
Transform: 3.23 seconds.
Transform: 3.28 seconds.
Transform: 3.24 seconds.
Transform: 3.36 seconds.
Transform: 3.26 seconds.
Transform: 3.36 seconds.
Transform: 3.21 seconds.
Transform: 3.32 seconds.
Transform: 3.24 seconds.
Transform: 3.30 seconds.
Transform: 3.29 seconds.
Transform: 3.46 seconds.
Transform: 3.25 seconds.
Transform: 3.38 seconds.
Transform: 3.29 seconds.
Transform: 3.42 seconds.
Transform: 3.29 seconds.
Transform: 3.51 seconds.
Transform: 3.23 seconds.
Transform: 3.29 seconds.
Transform: 3.24 seconds.
Transform: 3.34 seconds.
Transform: 3.22 seconds.
Transform: 3.28 seconds.
Transform: 3.29 seconds.
Transform: 3.45 seconds.
Transform: 3.28 seconds.
Transform: 3.44 seconds.


# Read

In [6]:
warp_setting = 1
trial_idx = 0

In [7]:
# Read transformed volumes

structure = '7N_L'

warped_volume = DataManager.load_transformed_volume(stack_m=stack_moving, 
                                                                          stack_f=stack_fixed, 
                                    classifier_setting_m=classifier_setting,
                                    classifier_setting_f=classifier_setting,
                                    warp_setting=warp_setting,
                                    trial_idx=0, structure=structure)

In [ ]:
display_volume_sections(warped_volume)

# Structure-wise Transform

In [5]:
classifier_setting = 2
warp_setting = 5
trial_idx = 0

In [7]:
if warp_setting == 1:
    upstream_warp_setting = None
elif warp_setting == 2:
    upstream_warp_setting = 1
elif warp_setting == 4:
    upstream_warp_setting = 1
elif warp_setting == 5:
    upstream_warp_setting = 1
else:
    raise Exception('Warp setting not recognized.')
    
if trial_idx in [0, 1]:
    upstream_trial_idx = 0

In [8]:
for structure in all_known_structures_sided:

    # Load local transform parameters
    
    try:

        t = time.time()
        
        local_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
        DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                              classifier_setting_m=classifier_setting,
                                              classifier_setting_f=classifier_setting,
                                              warp_setting=warp_setting,
                                              param_suffix=structure,
                                              trial_idx=trial_idx)

        # Read global tx
        global_transformed_moving_structure_vol = \
        DataManager.load_transformed_volume(stack_m=stack_moving, stack_f=stack_fixed,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            warp_setting=upstream_warp_setting, 
                                            trial_idx=upstream_trial_idx,
                                            structure=structure)

        # Transform
        local_transformed_moving_structure_vol = transform_volume(vol=global_transformed_moving_structure_vol, 
                                                 global_params=local_params, 
                                                 centroid_m=centroid_m, centroid_f=centroid_f,
                                                 xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

        # Save
        local_transformed_moving_structure_fn = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving, stack_f=stack_fixed,
                                                    classifier_setting_m=classifier_setting,
                                                    classifier_setting_f=classifier_setting,
                                                    warp_setting=warp_setting,
                                                    trial_idx=trial_idx,
                                                    structure=structure)

        create_if_not_exists(os.path.dirname(local_transformed_moving_structure_fn))
        bp.pack_ndarray_file(local_transformed_moving_structure_vol, local_transformed_moving_structure_fn)
        
        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t))
        
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error transforming volume %s.\n' % structure)

Transform: 5.52 seconds.
Transform: 5.47 seconds.
Transform: 5.48 seconds.
Transform: 5.48 seconds.
Transform: 5.51 seconds.
Transform: 5.52 seconds.
Transform: 5.52 seconds.
Transform: 5.54 seconds.
Transform: 5.56 seconds.
Transform: 5.50 seconds.
Transform: 5.48 seconds.
Transform: 5.50 seconds.
Transform: 5.51 seconds.
Transform: 5.50 seconds.
Transform: 5.52 seconds.
Transform: 5.52 seconds.
Transform: 5.50 seconds.
Transform: 5.46 seconds.
Transform: 5.51 seconds.
Transform: 5.53 seconds.
Transform: 5.45 seconds.
Transform: 5.48 seconds.
Transform: 5.53 seconds.
Transform: 5.50 seconds.
Transform: 5.65 seconds.
Transform: 5.62 seconds.
Transform: 5.48 seconds.
Transform: 5.50 seconds.
Transform: 5.51 seconds.
Transform: 5.47 seconds.
Transform: 5.58 seconds.
Transform: 5.56 seconds.
Transform: 5.57 seconds.
Transform: 5.56 seconds.
Transform: 5.66 seconds.
Transform: 5.69 seconds.
Transform: 5.53 seconds.
Transform: 5.45 seconds.
Transform: 5.51 seconds.
Transform: 5.46 seconds.


# Polyrigid Transform

In [3]:
warp_setting = 12
upstream_warp_setting = 1
classifier_setting = 2

In [4]:
globally_registered_volume = \
DataManager.load_volume_all_known_structures(stack_m=stack_moving, stack_f=stack_fixed,
                                            warp_setting=upstream_warp_setting,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            trial_idx=0)

File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0/score_volumes/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0_sp5.bp
[Errno 2] No such file or directory: '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0/score_volumes/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0_sp5.bp'
Score volume for sp5 does not exist.
File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0/score_volumes/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0_outerContour.bp
[Errno 2] No such file or directory: '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_tri

In [5]:
structure_sizes = {k: np.count_nonzero(v) for k, v in globally_registered_volume.iteritems()}

total_size = sum(structure_sizes.values())
structure_sizes_percent = {s: float(size)/total_size for s, size in structure_sizes.iteritems()}

In [6]:
structure_covars = {}
for s, v in globally_registered_volume.iteritems():
    ys, xs, zs = np.where(v)
    nzs = np.c_[xs, ys, zs]
    nzsc = nzs - nzs.mean(axis=0)
    C = np.dot(nzsc.T, nzsc)/float(len(nzsc))
    S, V = np.linalg.eigh(C)
    structure_covars[s] = C

In [7]:
# Read Transform of each structure, do polyrigid transform

rigid_parameters = {}
anchor_points = {}

for structure in all_known_structures_sided:

    try:
        tx_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
        DataManager.load_alignment_parameters(stack_m=stack_moving,
                                          stack_f=stack_fixed,
                                          classifier_setting_f=2,  
                                          classifier_setting_m=2,
                                        warp_setting=5,
                                           trial_idx=0,
                                            param_suffix=structure)
        
        rigid_parameters[structure] = (tx_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f)
        anchor_points[structure] = centroid_m
    except:
        sys.stderr.write('Error loading transform parameters for %s.\n' % structure)

File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_registration_parameters/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_5_MD642_down32_scoreVolume_clf_2_trial_0/atlasV2_down32_scoreVolume_clf_2_warp_5_MD642_down32_scoreVolume_clf_2_trial_0_parameters_sp5.txt
Error loading transform parameters for sp5.
File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_registration_parameters/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_5_MD642_down32_scoreVolume_clf_2_trial_0/atlasV2_down32_scoreVolume_clf_2_warp_5_MD642_down32_scoreVolume_clf_2_trial_0_parameters_outerContour.txt
Error loading transform parameters for outerContour.


In [8]:
weights = structure_sizes_percent
alpha = 1.
sigmas = {s: alpha * v for s, v in structure_covars.iteritems()}

In [ ]:
# Parallel

def func(structure):
    
    polyrigid_registered_volume = transform_volume_polyrigid(globally_registered_volume[structure], 
                                  rigid_parameters.values(), 
                                  anchor_points.values(), 
                                  sigmas.values(), 
                                  weights.values())

    # Save
    fn = \
    DataManager.get_transformed_volume_filepath(stack_m=stack_moving, stack_f=stack_fixed,
                                                classifier_setting_m=classifier_setting,
                                                classifier_setting_f=classifier_setting,
                                                warp_setting=warp_setting,
                                                trial_idx=0,
                                                structure=structure)

    create_parent_dir_if_not_exists(fn)
    bp.pack_ndarray_file(polyrigid_registered_volume, fn)

In [ ]:
pool = Pool(15)
pool.map(func, globally_registered_volume.keys())
pool.terminate()
pool.join()

In [17]:
# Sequential

for structure, gv in globally_registered_volume.iteritems():
    print structure
    polyrigid_registered_volume = transform_volume_polyrigid(gv, 
                                  rigid_parameters.values(), 
                                  anchor_points.values(), 
                                  sigmas.values(), 
                                  weights.values())

    # Save
    fn = \
    DataManager.get_transformed_volume_filepath(stack_m=stack_moving, stack_f=stack_fixed,
                                                classifier_setting_m=classifier_setting,
                                                classifier_setting_f=classifier_setting,
                                                warp_setting=warp_setting,
                                                trial_idx=0,
                                                structure=structure)

    create_parent_dir_if_not_exists(fn)
    bp.pack_ndarray_file(polyrigid_registered_volume, fn)

LRt_R
Sp5C_L
Pn_L
7N_R
PBG_L
VLL_L
4N_R
VLL_R
7N_L
PBG_R
RtTg
Sp5C_R
Pn_R
LRt_L
Sp5O_R
SNR_L
12N
SNR_R
Sp5O_L
DC_R
VCA_L
3N_L
6N_R
Sp5I_L
RMC_R
5N_L
SNC_L
SNC_R
AP
RMC_L
Sp5I_R
3N_R
6N_L
IC
VCA_R
DC_L
Amb_R
VCP_R
LC_R
10N_R
7n_R
5N_R
Tz_L
Tz_R
7n_L
4N_L
LC_L
10N_L
SC
VCP_L
Amb_L


# Global Transform Reversed

In [4]:
warp_setting = 3
trial_idx = 0

In [6]:
if warp_setting == 1:
    upstream_warp_setting = None
elif warp_setting == 2:
    upstream_warp_setting = 1
elif warp_setting == 3:
    global_warp_setting = 1
    upstream_warp_setting = 2
else:
    raise Exception('Warp setting not recognized.')
    
if trial_idx in [0, 1]:
    upstream_trial_idx = 0

In [7]:
# Load transform parameters

global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                      classifier_setting_m=classifier_setting,
                                      classifier_setting_f=classifier_setting,
                                      warp_setting=global_warp_setting,
                                      trial_idx=trial_idx)

In [ ]:
# for structure in ['PBG_L']:
for structure in all_known_structures_sided:
    
    try:
        
        t = time.time()

        local_transformed_moving_structure_vol = \
            DataManager.load_transformed_volume(stack_m=stack_moving, stack_f=stack_fixed,
                                                classifier_setting_m=classifier_setting,
                                                classifier_setting_f=classifier_setting,
                                                warp_setting=upstream_warp_setting, 
                                                trial_idx=upstream_trial_idx,
                                                structure=structure)

        local_transformed_moving_structure_global_reversed_vol = \
        transform_volume_inverse(vol=local_transformed_moving_structure_vol, 
                                 global_params=global_params, 
                                 centroid_m=centroid_m, centroid_f=centroid_f,
                                 xdim_m=xdim_m, ydim_m=ydim_m, zdim_m=zdim_m)

        local_transformed_moving_structure_global_reversed_fn = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='score',
                                                        stack_f=stack_fixed, type_f='score',
                                                        downscale=32,
                                                        classifier_setting_m=classifier_setting,
                                                        classifier_setting_f=classifier_setting,
                                                        warp_setting=warp_setting,
                                                        structure=structure)

        create_if_not_exists(os.path.dirname(local_transformed_moving_structure_global_reversed_fn))
        bp.pack_ndarray_file(local_transformed_moving_structure_global_reversed_vol, local_transformed_moving_structure_global_reversed_fn)

        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t))
        
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error transforming volume %s.\n' % structure)